In [4]:
import pandas as pd

from surprise import SVD, SVDpp, CoClustering
from surprise import NormalPredictor

from surprise import Dataset
from surprise import Reader

from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

from helpers import *

In [5]:
data = load_data("data/data_train.csv")
data.head()

,user,movie,rating
0,r44,c1,4
1,r61,c1,3
2,r67,c1,4
3,r72,c1,3
4,r86,c1,5


In [6]:
dataset = Dataset.load_from_df(data[['user', 'movie', 'rating']], Reader(rating_scale=(1, 5)))

In [14]:
param_grid = {'n_epochs': [20, 20], 'n_cltr_u': [3, 3],
              'n_cltr_i': [20, 40]}
gs = GridSearchCV(CoClustering, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(dataset)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

1.0063599718830543
{'n_epochs': 20, 'n_cltr_u': 3, 'n_cltr_i': 40}


In [15]:
algo = gs.best_estimator['rmse']

# Run 5-fold cross-validation and print results.
cross_validate(algo, dataset, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0039  1.0076  1.0026  1.0010  1.0014  1.0033  0.0024  
MAE (testset)     0.8042  0.8081  0.8038  0.8031  0.8038  0.8046  0.0018  
Fit time          64.49   67.63   67.67   67.26   69.13   67.24   1.51    
Test time         1.71    2.23    2.41    2.17    2.43    2.19    0.26    


{'test_rmse': array([1.00389016, 1.0076488 , 1.0025931 , 1.00095111, 1.0014382 ]),
 'test_mae': array([0.80416979, 0.80809058, 0.80384081, 0.80312238, 0.80382628]),
 'fit_time': (64.4925172328949,
  67.63182997703552,
  67.67201685905457,
  67.26323795318604,
  69.13187313079834),
 'test_time': (1.7133100032806396,
  2.2275898456573486,
  2.4125359058380127,
  2.170027017593384,
  2.4257969856262207)}

In [16]:
gen_submission(r'data/submission_coclustering3.csv', algo)